<a href="https://colab.research.google.com/github/JunHyeong-data/ML-DL-Study/blob/main/Basic-Deep-Learning/18_CNN_%EB%B0%B0%EC%B9%98_%EC%A0%95%EA%B7%9C%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN에서의 Batch Normalization

## 1. Batch Normalization 복습

딥러닝 모델은 일반적으로 **배치 단위 학습**을 사용한다.  
또한 매우 깊은 구조를 가지며, 각 레이어마다 **Activation Function**이 적용된다.

### 문제점
- Activation 이전의 입력 분포가
  - 모두 0보다 크거나
  - 모두 0보다 작은 경우
- 뉴런이 비활성화되거나 포화되어 **학습이 느려지거나 불안정**해진다.

### 해결 아이디어
- 입력 분포를 **0 근처의 정규분포**로 맞춰주자
- 이를 위해 배치 단위로
  - 평균(mean)
  - 분산(variance)
  를 계산한다.

---

## 2. Batch Normalization의 핵심 요소

### 정규화 과정
1. 배치 기반 평균 계산
2. 배치 기반 분산 계산
3. 정규화 수행

### 학습 가능한 파라미터
- **γ (gamma)**: 분포의 스케일 조절
- **β (beta)**: 분포의 이동 조절

> 모든 feature가 동일한 정규분포를 가지면 학습 의미가 없어지므로  
> γ, β를 통해 각 feature마다 다른 분포를 학습 가능하게 만든다.

### 추가 요소
- **Moving Average (이동 평균)**  
  → 학습 시뿐 아니라 전체 데이터 분포를 반영하기 위함

---

## 3. Fully Connected Layer에서의 Batch Normalization

- 입력 형태: `(Batch, Feature)`
- **Feature 단위**로 평균과 분산 계산
- 각 feature마다 γ, β 학습

➡ 이는 **Linear Neural Network**에 적용되는 BatchNorm 구조이다.

---

## 4. CNN에서의 Batch Normalization 필요성

CNN은 입력 텐서가 다음과 같은 구조를 가진다.

```

(Batch, Channel, Height, Width)

````

- 이미지에는 **공간 정보(Height, Width)** 가 존재
- FC Layer와 동일한 방식의 정규화는 부적절

➡ CNN에 맞는 Batch Normalization 방식이 필요하다.

---

## 5. CNN에서의 Batch Normalization 방식

### 정규화 기준
- **채널(Channel) 단위**로 정규화 수행
- 통계 계산에 포함되는 축:
  - Batch dimension
  - Height
  - Width

즉,
> 각 채널마다  
> (Batch × Height × Width) 전체를 기준으로  
> 평균과 분산을 계산한다.

### 결과
- 채널별로 평균과 분산 계산
- 채널별 γ, β 학습

---

## 6. FC BatchNorm vs CNN BatchNorm

| 구분 | 정규화 기준 |
|----|-----------|
| Fully Connected | Feature별 |
| CNN | Channel별 |

---

## 7. Activation Function과의 연결

1. Convolution 연산
2. Batch Normalization (채널별 정규화)
3. Activation Function 적용

➡ 정규화된 분포를 기반으로 안정적인 activation 출력 생성

---

## 8. PyTorch에서의 BatchNorm2d

PyTorch에서는 CNN용 BatchNorm을 다음과 같이 제공한다.

```python
nn.BatchNorm2d(num_features=채널수)
````

* 추가적인 인자 없이 **채널 수만 지정**
* 내부적으로 평균과 분산 자동 계산

### 실무 팁

* BatchNorm은 평균을 내부에서 계산하므로
* **이전 Convolution Layer의 bias는 불필요**
* → `bias=False` 권장

---

## 9. BatchNorm을 적용한 CNN 구조 예시

* Convolution Layer
* BatchNorm2d
* Activation
* (반복)
* Global Average Pooling
* Classification

➡ 이전보다 **더 빠르고 안정적인 학습** 가능

---

## 10. Batch Normalization 외의 Normalization 기법들

다양한 정규화 기법들이 존재한다.

* Batch Normalization
* Layer Normalization
* Instance Normalization
* Group Normalization
* 기타 변형 기법들

이 중,

* **Transformer 아키텍처**에서는
  **Layer Normalization**이 주로 사용된다.

(해당 내용은 Transformer 챕터에서 다룰 예정)

---

## 11. 정리

* Batch Normalization은 학습 안정성과 속도를 크게 개선한다.
* CNN에서는 **채널 단위 정규화**를 수행한다.
* FC Layer와 CNN에서의 BatchNorm은 **정규화 기준이 다르다**.
* BatchNorm2d는 현대 CNN에서 매우 널리 사용된다.

---

## 다음 내용

다음 시간에는
지금까지 배운 CNN 구성 요소들을 모두 활용하여
**실제 CNN 모델 코드를 구현**해본다.

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt

if torch.backends.mps.is_available():
    my_device = torch.device('mps')
elif torch.cuda.is_available():
    my_device = torch.device('cuda')
else:
    my_device = torch.device('cpu')

print(my_device)

cpu


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Load CIFAR10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=4)

100%|██████████| 170M/170M [00:03<00:00, 51.2MB/s]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [4]:
class ModernGAPCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),  # Batch normalization layer after first convolution
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False), # Stride 2 to reduce dimensions
            nn.BatchNorm2d(128),  # Batch normalization layer after second convolution
            nn.ReLU(inplace=True),
        )

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)  # Flatten the tensor before the fully connected layer
        x = self.classifier(x)
        return x
net = ModernGAPCNN(num_classes=10)


# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [5]:
net.to(my_device)
num_epochs = 100
for epoch in range(num_epochs):
    net.train()
    for batch_idx, (data, label) in enumerate(trainloader):
        data, label = data.to(my_device), label.to(my_device)
        scores = net(data)
        loss = criterion(scores, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    net.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, label in testloader:
            data, label = data.to(my_device), label.to(my_device)
            scores = net(data)
            loss = criterion(scores, label)
            val_loss += loss.item() * data.size(0)

            predicted = scores.argmax(dim=1)
            correct += predicted.eq(label).sum().item()

    val_loss /= len(testloader.dataset)
    val_accuracy = 100. * correct / len(testloader.dataset)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

Epoch [1/100], Training Loss: 1.6349, Validation Loss: 1.6332, Validation Accuracy: 41.62%


KeyboardInterrupt: 